In [1]:
from pointnetfunct.model_loader import vtp_to_point_cloud
import pointnetfunct.data_process_ml as data_process_ml
from pointnetfunct.PointNet_struct import PointNet,PointNet_2Branch,PointNet_3Branch
from pointnetfunct.PointNet_trainingfunct import load_model
import torch
import numpy as np
from torchvision import transforms

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
morpho_path = ".\AneuX\data-v1.0\data\morpho-per-cut.csv"
patient_path = ".\AneuX\data-v1.0\data\clinical.csv"
morpho_data_patient = data_process_ml.read_and_combine_data(morpho_path,patient_path)
merged_dataset = data_process_ml.encode_column(morpho_data_patient)
merged_dataset = data_process_ml.drop_columns(merged_dataset)
morpho_data_cut1,morpho_data_dome = data_process_ml.output_cut1anddome(merged_dataset)

morpho_data_cut1 = merged_dataset[merged_dataset["cuttype"] == "cut1"]
morpho_data_cut1.drop(morpho_data_cut1.columns[3:23], axis=1, inplace=True)
morpho_data_cut1.drop(['source_x',"cuttype"], axis=1, inplace=True)

morpho_data_dome = merged_dataset[merged_dataset["cuttype"] == "dome"]
morpho_data_dome.drop(['source_x',"cuttype"], axis=1, inplace=True)
#morpho_data_dome.set_index("dataset")


raw_data = morpho_data_dome
#raw_data.drop(["dataset"], axis=1, inplace=True)
#raw_data = morpho_data_dome.astype(float)

raw_data.loc[raw_data['sex_male'] == 0, 'sex_male'] = 2
#print(self.raw_data.iloc[0])
#raw_data.drop(("status_ruptured"),axis=1,inplace=True)
raw_data.drop(("sex_female"),axis=1,inplace=True)
raw_data['age'].fillna(raw_data['age'].mean(), inplace=True)

d:\Universityofleeds\MSC_PROG\Msc_project\code\pointnetfunct\data_process_ml.py:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  morpho_data_cut1.drop(morpho_data_cut1.columns[3:23], axis=1, inplace=True)
d:\Universityofleeds\MSC_PROG\Msc_project\code\pointnetfunct\data_process_ml.py:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  morpho_data_cut1.drop(['source_x',"cuttype","dataset"], axis=1, inplace=True)
d:\Universityofleeds\MSC_PROG\Msc_project\code\pointnetfunct\data_process_ml.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation

In [3]:
def pointnet_testcase(model_path,IA_parameters,model,label):
    pointnet_copy = PointNet_2Branch(classes=2)

    load_model(pointnet_copy, model_path)    
    y_true = []
    y_pred = []
    y_pred_result = []
    y_probs = []
    pointnet_copy = pointnet_copy.to("cpu")
    with torch.no_grad():
        transform = transforms.ToTensor()
        inputs_v = model
        inputs_v = np.asarray(inputs_v.points)
        inputs_v = transform(inputs_v)

        print(inputs_v)
        inputs_v3 = torch.from_numpy(np.array(IA_parameters,dtype = np.float64))
        
        inputs_v = inputs_v.to(torch.float32)
        inputs_v = inputs_v.squeeze(1).permute(0, 2, 1)
        inputs_v3 = inputs_v3.to(torch.float32)
        pointnet_copy.eval()

        output = pointnet_copy.forward(inputs_v,inputs_v3)
        
        #print(torch.exp(outputs))

        outputs_value = (torch.max(torch.exp(output), 1)[0]).data.cpu().numpy()
        outputs_result = (torch.max(torch.exp(output), 1)[1]).data.cpu().numpy()
        
        
        #print(outputs)
        y_pred.extend(outputs_value)
        y_pred_result.extend(outputs_result)
            
        y_true.extend(label)
        y_probs.extend(output)
            
    return y_true,y_pred,y_pred_result,y_probs

In [10]:
modelcov_vtp = vtp_to_point_cloud("./test_model/C0014_cut1.vtp",points=2000)
model_path = "./final_models/2000points/cut1/pointnet_cut1_2branch_uniform_2000pt.pth"
model_name = "C0014"
row = raw_data[raw_data["dataset"] == model_name]
row.drop(["dataset"], axis=1, inplace=True)
label = row["status_ruptured"]
row.drop(("status_ruptured"),axis=1,inplace=True)
row = row.astype(float)
pointnet_testcase(model_path,row,modelcov_vtp,label)

tensor([[[36.9034, 35.2812, 20.7851],
         [40.0638, 33.5709, 18.9951],
         [37.1639, 37.5233, 20.1238],
         ...,
         [35.9780, 33.7617, 23.7320],
         [38.0551, 36.0254, 15.5611],
         [41.7114, 30.1818, 23.1558]]], dtype=torch.float64)


C:\Users\eulershadow\AppData\Local\Temp\ipykernel_1624\3564381691.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  row.drop(["dataset"], axis=1, inplace=True)
C:\Users\eulershadow\AppData\Local\Temp\ipykernel_1624\3564381691.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  row.drop(("status_ruptured"),axis=1,inplace=True)


([0.0], [0.80120206], [0], [tensor([-0.2216, -1.6155])])

In [23]:
from pointnetfunct.model_loader import vtp_to_mesh,vtp_to_point_cloud
import open3d as o3d
#mesh_vessel_vtp = vtp_to_point_cloud("./test_model/SNF00000614.vtp", points= 2000)
mesh_vessel_vtp =  o3d.io.read_triangle_mesh("./test_model/SNF00000614_cut1.stl")
mesh_vessel_vtp = vtp_to_mesh("./test_model/C0014_cut1.vtp")
o3d.visualization.draw_geometries([mesh_vessel_vtp])

In [24]:

# Create a visualizer
vis = o3d.visualization.Visualizer()
vis.create_window()

# Add the mesh to the visualizer
vis.add_geometry(mesh_vessel_vtp)

# Get the render options and enable lighting
opt = vis.get_render_option()
opt.light_on = False
opt.mesh_show_back_face = False


# Run the visualizer
vis.run()
vis.destroy_window()
